In [ ]:
import json
import numpy as np
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

#assuming, we've already trained the models, and the models are available and utilizable in the filesystem -- let's put the address of where the files are in! That was we can use them.
CATEGORICAL_CLASSIFIER_MODEL_PATH = 'categorical_classifier.h5'
INTENSITY_PREDICTOR_MODEL_PATH = 'intensity_predictor.h5'

#Okay we have two models, one to predict pre-determined categories, and on to predict intensity of the queries.
categorical_classifier = keras.models.load_model(CATEGORICAL_CLASSIFIER_MODEL_PATH)
intensity_predictor = keras.models.load_model(INTENSITY_PREDICTOR_MODEL_PATH)


#This just serves to tokenize each query, make each query the same length as the other queries, remove punctuation, etc.
def preprocess_query(query, tokenizer, max_length=100):
    #use the trained tokenizer, provided as a parameter in the function.
    sequences = tokenizer.texts_to_sequences([query])
    #pad the sequences to the right length.
    padded = pad_sequences(sequences, maxlen=max_length)
    return padded


def predict_category_and_intensity(json_input):
    #load in the data from the JSON. We have JSON, but we want the string (the actual query) that the data represents
    data = json.loads(json_input)
    query = data['query']

    tokenizer = Tokenizer(num_words=30000) #Since we are generating about 1000 queries, we can assume 30-ish words per query? If each word (worst-case) is distinct, the tokenizer needs a dictionary of 30000 words.

    #let's call the function above!
    processed_query = preprocess_query(query, tokenizer)

    #below, we are passing the query into both models, taking the output, storing the predicted category in cateogry_pred and the predicted intensity in intensity_pred.
    category_pred = categorical_classifier.predict(processed_query), axis
    intensity_pred = np.argmax(intensity_predictor.predict(processed_query), axis=-1)

    #We are returning a list of {x, y} where x is the predicted cateogry, and y is the predicted intesity
    return {
        "category_prediction": str(category_pred),
        "intensity_prediction": int(intensity_pred)
    }
